In [1]:
import PyKDL as kdl
import kdl_parser_py.urdf
import pickle
import torch
import sys, os
import cv2 as cv
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import transformations as tf
import numpy as np
from itertools import repeat

np.set_printoptions(suppress=True)

%load_ext autoreload
%autoreload 2

util_path = os.path.abspath("../utils/")
sys.path.append(util_path)
import transformation as trans
import projections as proj
import robot_visualize as rbvis



In [3]:
act_path = "/home/zfei/code/act-plus-plus/"
sys.path.append(act_path)
from policy import EPACTPolicy, ACTPolicy
from brl_constants import TASK_CONFIGS
ckpt_path = "/mnt/data1/act/train_act_scara_3cam/3cam_epact_v4_1_lep10/"

config_path = ckpt_path + "config.pkl"
# step = 500000
# policy_model_path = ckpt_path + "policy_step_%i_seed_0.ckpt"%step
policy_model_path = ckpt_path + "policy_step_30000_seed_0.ckpt"

stats_path = ckpt_path + "dataset_stats.pkl"

with open(config_path, "rb") as f:
    policy_config = pickle.load(f)['policy_config']
    policy_config["ep_weight"] = 1.0
    print(policy_config)

policy = EPACTPolicy(policy_config)
# policy = ACTPolicy(policy_config)
policy_class = "EPACT"
# policy_class = "ACT"


pre_process_qpos = (
    lambda s_qpos: torch.from_numpy((s_qpos - stats["qpos_mean"]) / stats["qpos_std"])
    .float()
    .cuda()
    .unsqueeze(0)
)
post_process_action = lambda a: a * stats["action_std"] + stats["action_mean"]

print("loading EPACT policy success")

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data1/act/train_act_scara_3cam/3cam/config.pkl'

In [83]:
camera_names = policy_config["camera_names"]
print(camera_names)

['wrist', 'wrist_down', 'top']


In [84]:
def pre_proccess_img(img):
    img_torch = torch.from_numpy(img).unsqueeze(0)
    img_torch = torch.einsum("k h w c -> k c h w", img_torch)
    img_torch = (img_torch / 255.0).float().cuda().unsqueeze(0)
    return img_torch

def pre_process_multi_img(curr_images):
    # a list of images [w, h, 3]
    curr_image = np.stack(curr_images, axis=0)
    img_torch = torch.from_numpy(curr_image)
    img_torch = torch.einsum("k h w c -> k c h w", img_torch)
    img_torch = (img_torch / 255.0).float().cuda().unsqueeze(0)
    return img_torch

def post_process_all_actions(all_actions):
    # post process all actions
    joint_states_traj = []
    actual_actions = []
    for i, raw_action in enumerate(all_actions):
        actual_action = post_process_action(raw_action)
        actual_actions.append(actual_action)
    actual_actions = np.asarray(actual_actions)
    # print(actual_actions)
    return actual_actions


In [85]:
act_path = "/home/zfei/code/act-plus-plus/"
sys.path.append(act_path)
from utils import EpisodicDataset
from utils import find_all_hdf5, flatten_list, get_norm_stats
from utils import load_data # data functions

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:

def forward_pass_epact(data, policy):
    image_data, qpos_data, action_data, is_pad, end_pose_data = data
    image_data, qpos_data, action_data, is_pad, end_pose_data = image_data.cuda(), qpos_data.cuda(), action_data.cuda(), is_pad.cuda(), end_pose_data.cuda()
    return policy(qpos_data, image_data, action_data, is_pad, end_pose_data) # TODO remove None

def forward_pass(data, policy):
    image_data, qpos_data, action_data, is_pad = data
    image_data, qpos_data, action_data, is_pad = image_data.cuda(), qpos_data.cuda(), action_data.cuda(), is_pad.cuda()
    return policy(qpos_data, image_data, action_data, is_pad) # TODO remove None

def repeater(data_loader):
    epoch = 0
    for loader in repeat(data_loader):
        for data in loader:
            yield data
        print(f'Epoch {epoch} done')
        epoch += 1

In [87]:
dataset_path_list = find_all_hdf5("/mnt/data1/act/train_act_scara_3cam/data1/", skip_mirrored_data=False)
norm_stats, all_episode_len = get_norm_stats(dataset_path_list)

camera_names = ['wrist', 'wrist_down', 'top']
train_episode_ids = [0, 1]
train_episode_len = [all_episode_len[i] for i in train_episode_ids]

val_episode_ids = [0, 1]
val_episode_len = [all_episode_len[i] for i in val_episode_ids]

chunk_size = 64


Found 51 hdf5 files


In [88]:
dataset_dir = "/mnt/data1/act/train_act_scara_3cam/data1/"

TASK_CONFIGS = {
        'epact':{
        'dataset_dir': dataset_dir + '/train_act_scara_3cam',
        'num_episodes': 100,
        'episode_len': 300,
        'camera_names': ['wrist', 'wrist_down', 'top']
    }
}

In [89]:
task_config = TASK_CONFIGS['epact']
name_filter = task_config.get('name_filter', lambda n: True)
batch_size_train = 8
batch_size_val = 8


In [90]:
chunk_size = 100
skip_mirrored_data = False
load_pretrain = False
train_ratio = 0.99
train_dataloader,val_dataloader, stats, _ = load_data(dataset_dir, 
                                                       name_filter, 
                                                       camera_names, 
                                                       batch_size_train, 
                                                       batch_size_val, 
                                                       chunk_size,
                                                       skip_mirrored_data,
                                                       load_pretrain,
                                                       policy_class,
                                                       stats_dir_l=None,
                                                       sample_weights=None,
                                                       train_ratio=train_ratio)
train_dataloader = repeater(train_dataloader)


Found 51 hdf5 files


Data from: ['/mnt/data1/act/train_act_scara_3cam/data1/']
- Train on [50] episodes
- Test on [1] episodes


Found 51 hdf5 files
Norm stats from: ['/mnt/data1/act/train_act_scara_3cam/data1/']
train_episode_len: [120, 108, 118, 104, 140, 135, 136, 177, 129, 117, 168, 135, 144, 175, 125, 106, 111, 132, 121, 103, 154, 124, 153, 137, 168, 128, 101, 110, 124, 110, 128, 161, 151, 127, 116, 135, 114, 104, 118, 123, 90, 107, 118, 139, 117, 116, 98, 136, 165, 121], val_episode_len: [154], train_episode_ids: [36  5 11  3 50 31 17 44  9 48 19 29 45 25 18 22 12 15  2 30 10 28 41  0
 34 16 27 24 23 26  4 37 32  1 42  8 20 47 13 43 21  6 33 38 35 39  7 14
 49 46], val_episode_ids: [40]
val data:  /mnt/data1/act/train_act_scara_3cam/data1/episode_9.hdf5
URDF Path:  /home/zfei/code/act-plus-plus/urdf/hitbot_model.urdf
kdl_parse urdf ok?:  True
augment_images:  False
Initializing transformations
URDF Path:  /home/zfei/code/act-plus-plus/urdf/hitbot_model.urdf
kdl_parse urdf ok?:  

In [91]:
data = next(train_dataloader)
forward_pass_epact(data, policy)



{'l1': tensor(0.4697, device='cuda:0', grad_fn=<MeanBackward0>),
 'kl': tensor(9.8818, device='cuda:0', grad_fn=<SelectBackward0>),
 'ep': tensor(0.1555, device='cuda:0', grad_fn=<MeanBackward0>),
 'loss': tensor(99.4427, device='cuda:0', grad_fn=<AddBackward0>)}

In [92]:
policy.kl_weight

10

In [67]:
print("image_data", image_data.shape)
print("qpos_data", qpos_data.shape)
print("action_data", action_data.shape)
print("is_pad", is_pad.shape)

image_data torch.Size([8, 3, 3, 480, 640])
qpos_data torch.Size([8, 4])
action_data torch.Size([8, 100, 5])
is_pad torch.Size([8, 100])


In [68]:
policy.model.num_queries

100

In [69]:
policy(qpos_data, image_data, action_data, is_pad)

image torch.Size([8, 3, 3, 480, 640])
qpos torch.Size([8, 4])
actions torch.Size([8, 100, 5])
is_pad torch.Size([8, 100])


{'l1': tensor(0.7795, device='cuda:0', grad_fn=<MeanBackward0>),
 'kl': tensor(7.9838, device='cuda:0', grad_fn=<SelectBackward0>),
 'loss': tensor(80.6176, device='cuda:0', grad_fn=<AddBackward0>)}

In [13]:
h5data_file = "/mnt/data1/act/train_act_scara_3cam/data1/episode_30.hdf5"

with torch.inference_mode():
    with h5py.File(h5data_file, "r") as root:
        index = 0
        qpos_np = root["/observations/qpos"][index]
        img_wrist_up_np = root["/observations/images/wrist"][index]
        img_wrist_down_np = root["/observations/images/wrist_down"][index]
        img_top_np = root["/observations/images/top"][index]

        qpos_all =  root["/observations/qpos"][()]

        curr_images = [img_wrist_up_np, img_wrist_down_np, img_top_np]

        action_obs_np = root["action"][()]
        qpos = pre_process_qpos(qpos_np)
        curr_images_torch = pre_process_multi_img(curr_images)
        # print("qpos: ",qpos.shape)
        all_actions, all_end_poses = policy(qpos, curr_images_torch)
        all_actions = all_actions.squeeze(0).cpu().numpy()

        print("all_actions: ", all_actions.shape)
        print("action_obs_np: ", action_obs_np.shape)

        # actual_action = post_process_action(raw_action)
        # print("actual action %i"%index, actual_action)


action_obs_np = action_obs_np[index:]
qpos_all = np.asarray(qpos_all)
qpos_all = qpos_all[index:]


all_actions:  (100, 5)
action_obs_np:  (127, 5)
